In [58]:
# SETUP MAGIC
f = open("train.tsv")
cur = -1
all_X = []
all_y = []
for line in f:
    if cur == -1: # Skip first line
        cur += 1
        continue
    parts = line.split('\t')
    if cur < int(parts[1]):
        all_X.append(parts[2])
        all_y.append(int(parts[3]))
        cur += 1

# Split up 0-1: negative, 2(neutral) - skip, 3-4: positive
all_X2 = []
all_y2 = []
all_bad = []
all_good = []
for i in range(len(all_X)):
    if all_y[i] == 2:
        continue
    
    new_y = -1 if all_y[i] < 2 else 1
    
    all_y2.append(new_y)
    all_X2.append(all_X[i])
    
    if new_y == -1:
        all_bad.append(all_X[i])
    else:
        all_good.append(all_X[i])

In [64]:
# A bad example
print "Negative:"
print all_bad[1]
print
# A good one
print "Positive:"
print all_good[9]

Negative:
Even fans of Ismail Merchant 's work , I suspect , would have a hard time sitting through this one .

Positive:
Best indie of the year , so far .


In [86]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=5, stop_words='english', ngram_range=(1, 3))
vX = vectorizer.fit_transform(all_X2)

print "In human:", all_X2[20]
print "In computer:", vX[20].toarray()

In human: It 's like going to a house party and watching the host defend himself against a frothing ex-girlfriend .
In computer: [[0 0 0 ..., 0 0 0]]


In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vX, all_y2, test_size=0.2)
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [90]:
# How well we are doing:
print clf.score(X_train, y_train) * 100
print clf.score(X_test, y_test) * 100

92.8286129267
75.0181554103


In [91]:
# Let's look at some specific examples
testX = ["I have never seen a movie so dull and uninspired", "What a masterpiece, everyone must see it"]
textVX = vectorizer.transform(testX)
clf.predict(textVX)

array([-1,  1])

In [96]:
# What does it mean: -- try to do relative thing instead
sorted(zip(clf.coef_.toarray()[0], vectorizer.vocabulary_))[:100]

[(-2.0, u'throws'),
 (-1.9330584283740495, u'knowledge'),
 (-1.8237252052965494, u'rise'),
 (-1.7646188592245933, u'acts'),
 (-1.721978999336383, u'context'),
 (-1.7149508625620624, u'thoroughly'),
 (-1.6403974956751637, u'memory'),
 (-1.6375962854532924, u'loves'),
 (-1.6019483007599626, u'scenes'),
 (-1.5673042301791544, u'little'),
 (-1.5561083401461144, u'superficial'),
 (-1.5525062412051105, u'gives'),
 (-1.5255936378909059, u'drawn'),
 (-1.5101434724014378, u'document'),
 (-1.478283017610742, u'searching'),
 (-1.4753025160569726, u'adams'),
 (-1.4646256949572312, u'starts'),
 (-1.4640396229186812, u'engaging'),
 (-1.4037116547267507, u'light'),
 (-1.3997989987162449, u'delightful'),
 (-1.3886864814341839, u'search'),
 (-1.3619214381118936, u'spite'),
 (-1.3526686384548321, u'possibly'),
 (-1.3450119250199826, u'values'),
 (-1.3254031682868259, u'heads'),
 (-1.3103035643135859, u'michael moore'),
 (-1.2979669844960788, u'ethnic'),
 (-1.2927429828120582, u'bits'),
 (-1.280656406314

In [105]:
good_words = []
for good_sent in all_good:
    good_words += good_sent.lower().split()

    bad_words = []
for bad_sent in all_bad:
    bad_words += bad_sent.lower().split()

from collections import Counter
good_counts = Counter(good_words)
bad_counts = Counter(bad_words)

In [117]:
# Find relatively impactful words
all_words = set(good_counts.keys()).union(set(bad_counts.keys()))

goodness = []
badness = []
for word in all_words:
    goodness.append((word, (good_counts.get(word, 0.)/(bad_counts.get(word, 1.)))))
    badness.append((word, (bad_counts.get(word, 0.)/(good_counts.get(word, 1.)))))

In [122]:
from operator import itemgetter

best_words = sorted(goodness, key=itemgetter(1))[-50:]
good_words = sorted(badness, key=itemgetter(1))[-50:]

# Some cool vizualization here?
print best_words
print 
print
print good_words

[('enjoyed', 10.0), ('vividly', 10.0), ('thoughtful', 10), ('haunting', 10), ('faith', 10), ('undeniably', 10.0), ('gem', 10.0), ('vibrant', 10.0), ('heartwarming', 10.0), ('polanski', 10), ('intense', 10), ('delicate', 11), ('delivers', 11), ('wonderfully', 11.0), ('thought-provoking', 11.0), ('force', 11), ('chilling', 11), ('absorbing', 11), ('extraordinary', 11), ('riveting', 12.0), ('imax', 12), ('means', 12), ('amazing', 12), ('delight', 12), ('heartfelt', 12), ('oddly', 12), ('meditation', 12), ('vivid', 13), ('terrific', 13), ('unique', 13), ('places', 13), ('treat', 14), ('form', 14), ('refreshing', 14.0), ('inventive', 15.0), ('provides', 15.0), ('somewhat', 16), ('brings', 17), ('portrait', 17), ('perfectly', 17.0), ('unexpected', 17), ('eyes', 20), ('warm', 21), ('captures', 21), ('engrossing', 22), ('wonderful', 24), ('remarkable', 24), ('touching', 27), ('solid', 33.0), ('powerful', 37.0)]


[('holes', 10.0), ('failure', 10), ('joke', 10.0), ('contrived', 10), ('inept', 1